In [1]:
import pandas as pd
from matplotlib import pyplot as plt

plt.style.use('seaborn-whitegrid')

In [2]:
data = pd.read_csv('../data/data.csv', parse_dates=['date'])

data['train'] = data['num'] + '-' + data['class']

data.head(2)

,date,days,places,price,num,class,weekday,day_name,weekend,train
0,2021-04-03,1,0.442857,3275.0,001А,Купе,5,Saturday,0,001А-Купе
1,2021-04-04,1,0.235714,3913.0,001А,Купе,6,Sunday,0,001А-Купе


In [3]:
grouped = data.groupby(['train', 'days'])['price'].mean().reset_index()\
    .pivot(index='train', columns='days', values='price')

grouped.head(2)

days,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
train,,,,,,,,,,,,,,,,,,,,,
001А-Купе,3042.617647,3081.150000,2993.452381,2971.619048,2884.000000,2852.075,2797.595238,2766.238095,2746.833333,2682.976744,...,2783.969697,2804.516129,2815.633333,2840.642857,2840.250000,2799.777778,2847.769231,2860.130435,2879.166667,2898.952381
001А-СВ,6163.615385,6205.333333,5865.261905,5943.880952,5801.926829,5888.100,5940.738095,6058.690476,6097.833333,6088.651163,...,6183.151515,6212.322581,6189.733333,6112.821429,6177.964286,6149.370370,6120.884615,6070.260870,6056.791667,6043.190476


In [4]:
grouped.shape

(144, 30)

In [6]:
trains = grouped.index.tolist()

ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.5, hspace=0.6)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    row = grouped.loc[train]
    ax.plot(row, c='C3', lw=2)
    ax.fill_between(row.index, row.values, row.min(), color='C3', alpha=0.15)
    
    ax.set_title(train)
    ax.set_xlabel('days to departure')
    ax.set_ylabel('avg. ticket price')
    ax.invert_xaxis()
    
for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/price_vs_days.pdf', transparent=True, bbox_inches='tight')
plt.close()

In [10]:
grouped = data.groupby(['train', 'days'])['places'].mean().reset_index()\
    .pivot(index='train', columns='days', values='places')

grouped.head(2)

days,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
train,,,,,,,,,,,,,,,,,,,,,
001А-Купе,0.382784,0.436905,0.498810,0.549830,0.613415,0.636786,0.675510,0.707143,0.726701,0.751163,...,0.882900,0.885023,0.890238,0.902296,0.897194,0.897090,0.904121,0.897205,0.902381,0.907483
001А-СВ,0.605569,0.627604,0.686756,0.704241,0.734566,0.760352,0.764137,0.781436,0.794457,0.799964,...,0.778172,0.782258,0.795052,0.798549,0.796596,0.813079,0.808293,0.829144,0.838867,0.873512


In [11]:
grouped.shape

(144, 30)

In [14]:
trains = grouped.index.tolist()

ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.5, hspace=0.6)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    row = grouped.loc[train]
    ax.plot(row, c='C2', lw=2)
    ax.fill_between(row.index, row.values, row.min(), color='C2', alpha=0.15)
    
    ax.set_title(train)
    ax.set_xlabel('days to departure')
    ax.set_ylabel('avg. vacant places')
    ax.invert_xaxis()
    
for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/places_vs_days.pdf', transparent=True, bbox_inches='tight')
plt.close()